In [49]:
import psycopg2
import pandas as pd
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk.corpus import stopwords
import unicodedata
import string
import numpy as np
import spacy
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [3]:
def retrieve_reviews(product):
    conn = psycopg2.connect(dbname='product_reviews', user='postgres', password='', host='localhost')
    cursor = conn.cursor()
    p_no = 0
    r_no = 0
    cursor.execute("select * from reviews where product_name = '{}'".format(product))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    conn.close()
    return df

In [6]:
df = retrieve_reviews('LG Electronics 65UJ6300 65-Inch 4K Ultra HD Smart LED TV (2017 Model)')

In [7]:
df.head(2)

,0,1,2,3,4,5,6,7,8,9,10
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,1,5.0 out of 5 stars,"September 22, 2017",Prime Customer,"TV Great, TruMotion Bad!",Now that I found the TruMotion setting so that...,Verified Purchase Helpful No...
1,2,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,2,5.0 out of 5 stars,"December 30, 2017",GMC,Hint: Turn OFF Smart Picture Mode,Hint: Turn off Smart Picture Mode and then tur...,Verified Purchase Helpful No...


In [8]:
columns = ['id','p_name','url','p_no','r_no','r_stars','r_date','r_name','r_title','r_text','r_comments']

In [9]:
df.columns = columns

In [10]:
df.head(2)

,id,p_name,url,p_no,r_no,r_stars,r_date,r_name,r_title,r_text,r_comments
0,1,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,1,5.0 out of 5 stars,"September 22, 2017",Prime Customer,"TV Great, TruMotion Bad!",Now that I found the TruMotion setting so that...,Verified Purchase Helpful No...
1,2,LG Electronics 65UJ6300 65-Inch 4K Ultra HD Sm...,https://www.amazon.com/LG-Electronics-65UJ6300...,1,2,5.0 out of 5 stars,"December 30, 2017",GMC,Hint: Turn OFF Smart Picture Mode,Hint: Turn off Smart Picture Mode and then tur...,Verified Purchase Helpful No...


In [11]:
df.groupby(['r_stars']).count()

,id,p_name,url,p_no,r_no,r_date,r_name,r_title,r_text,r_comments
r_stars,,,,,,,,,,
1.0 out of 5 stars,78,78,78,78,78,78,78,78,78,78
2.0 out of 5 stars,25,25,25,25,25,25,25,25,25,25
3.0 out of 5 stars,41,41,41,41,41,41,41,41,41,41
4.0 out of 5 stars,73,73,73,73,73,73,73,73,73,73
5.0 out of 5 stars,257,257,257,257,257,257,257,257,257,257


In [12]:
df_new = df.copy()
df_new = df_new[df_new['r_stars'].apply(lambda x: x in ['5.0 out of 5 stars','1.0 out of 5 stars'])]
df_new['rating'] = df_new['r_stars'].apply(lambda x: 1 if x == '5.0 out of 5 stars' else 0)
df_new = df_new[['rating','r_name','r_text']]
df_new.head(3)

,rating,r_name,r_text
0,1,Prime Customer,Now that I found the TruMotion setting so that...
1,1,GMC,Hint: Turn off Smart Picture Mode and then tur...
2,1,AncientGhost,Needed an upgrade for my 7yr old HDTV and deci...


In [42]:
 
nlp = spacy.load("en")
document = df_new.iloc[1]['r_text']
parsed_document = nlp(document)

In [54]:
print (parsed_document)

Hint: Turn off Smart Picture Mode and then turn the Energy Saver OFF.  With Smart Picture Mode on and no access to Energy Saver setting, the picture was very dull and drab.  Energy Saver reduces the backlight to save energy but at the cost of poor picture quality.  After making this change, the TV picture was very bright and sharp with great detail.


In [55]:
for num, sentence in enumerate(parsed_document.sents):
    print (num, sentence)

0 Hint: Turn off Smart Picture Mode and then turn the Energy Saver OFF.  With Smart Picture Mode on and no access to Energy Saver setting, the picture was very dull and drab.  
1 Energy Saver reduces the backlight to save energy but at the cost of poor picture quality.  
2 After making this change, the TV picture was very bright and sharp with great detail.


In [56]:
for num, entity in enumerate(parsed_document.ents):
    print (entity, entity.label_)
token_text = [token.orth_ for token in parsed_document]
token_pos = [token.pos_ for token in parsed_document]

the Energy Saver OFF. ORG
Energy Saver ORG


In [57]:
token_lemma = [token.lemma_ for token in parsed_document]
token_shape = [token.shape_ for token in parsed_document]
token_prob = [token.prob for token in parsed_document]

In [47]:
df_spicy = pd.DataFrame({'token_text':token_text, 'token_lemma':token_lemma, 'token_shape':token_shape,'token_prob':token_prob})

In [48]:
df_spicy

,token_lemma,token_prob,token_shape,token_text
0,hint,-12.858102,Xxxx,Hint
1,:,-6.128876,:,:
2,turn,-11.648442,Xxxx,Turn
3,off,-7.073742,xxx,off
4,smart,-12.219462,Xxxxx,Smart
5,picture,-12.162342,Xxxxx,Picture
6,mode,-12.467278,Xxxx,Mode
7,and,-4.113108,xxx,and
8,then,-6.598201,xxxx,then
9,turn,-8.652341,xxxx,turn


In [108]:
def punct_space(token):
    return token.is_punct or token.is_space

def lemmatized_sentence_corpus (documents):
    for doc in documents:
        parsed_doc = nlp(doc)
        for sent in parsed_doc.sents:
            yield (' '.join([token.lemma_ for token in sent if not punct_space(token)]))


In [164]:
bigram_model = Phrases([doc.split(" ") for doc in lemmatized_sentence_corpus(df_new['r_text'].values)])

In [139]:
bigram_model.vocab


defaultdict(int,
            {b'now': 20,
             b'that': 110,
             b'now_that': 3,
             b'i': 398,
             b'that_i': 10,
             b'find': 20,
             b'i_find': 6,
             b'the': 592,
             b'find_the': 4,
             b'trumotion': 5,
             b'the_trumotion': 4,
             b'set': 37,
             b'trumotion_set': 1,
             b'so': 88,
             b'set_so': 1,
             b'so_that': 2,
             b'could': 23,
             b'i_could': 10,
             b'turn': 27,
             b'could_turn': 1,
             b'it': 331,
             b'turn_it': 5,
             b'off': 29,
             b'it_off': 2,
             b'off_i': 1,
             b'really': 17,
             b'i_really': 3,
             b'like': 57,
             b'really_like': 2,
             b'this': 227,
             b'like_this': 3,
             b'tv.': 48,
             b'this_tv.': 14,
             b'tv._it': 2,
             b'take': 30,
             b'i

In [150]:
print (''.join(bigram_model['now', 'that']))

now_that


/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [166]:
bigram_sentences = []
for unigram_sentence in lemmatized_sentence_corpus(df_new['r_text'].values):
    bigram_sentences.append(' '.join(bigram_model[unigram_sentence.split(" ")]))

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [185]:
trigram_model = Phrases([doc.split(" ") for doc in bigram_sentences], min_count=1)

In [186]:
trigram_model.vocab

defaultdict(int,
            {b'now': 20,
             b'that': 110,
             b'now_that': 3,
             b'i': 398,
             b'that_i': 10,
             b'find': 20,
             b'i_find': 6,
             b'the': 592,
             b'find_the': 4,
             b'trumotion': 5,
             b'the_trumotion': 4,
             b'set': 14,
             b'trumotion_set': 1,
             b'so': 78,
             b'set_so': 1,
             b'so_that': 2,
             b'could': 23,
             b'i_could': 10,
             b'turn': 18,
             b'could_turn': 1,
             b'it': 331,
             b'turn_it': 5,
             b'off': 20,
             b'it_off': 2,
             b'off_i': 1,
             b'really': 17,
             b'i_really': 3,
             b'like': 57,
             b'really_like': 2,
             b'this': 227,
             b'like_this': 3,
             b'tv.': 48,
             b'this_tv.': 14,
             b'tv._it': 2,
             b'take': 30,
             b'i

In [169]:
# from gensim.models import Phrases
# documents = ["the mayor of york mayor was there", "machine learning can be useful sometimes","new york mayor was present"]

# sentence_stream = [doc.split(" ") for doc in documents]
# print (sentence_stream)
# bigram = Phrases(sentence_stream, min_count=1, threshold=1)
# sent = [u'the', u'of', u'new', u'york',  u'mayor', u'was', u'there']
# print(bigram[sent])

In [173]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings


In [174]:
len(bigram_sentences)

1187

In [187]:
trigram_sentences = []
for bigram_sentence in bigram_sentences:
    trigram_sentences.append(' '.join(trigram_model[bigram_sentence.split(" ")]))

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [176]:
len(trigram_sentences)

1187

In [193]:
trigram_sentences[2]

'the trumotion setting make 24fps movie_look like 30fps home movie otherwise know as soap_opera effect'

In [196]:
trigram_reviews = LineSentence(trigram_sentences)

In [214]:
trigram_dictionary = Dictionary([doc.split(" ") for doc in trigram_sentences])

In [207]:
trigram_dictionary.filter_extremes(no_below=5, no_above=0.4)

In [219]:
def trigram_bow_generator(trigram_sentences):
    trigram_bow = []
    for review in trigram_sentences:
        print (review)
        trigram_bow.append(trigram_dictionary.doc2bow(review))

In [223]:
trigram_bow = trigram_bow_generator([doc.split(" ") for doc in trigram_sentences])

['now', 'that', 'i', 'find', 'the', 'trumotion', 'set', 'so', 'that', 'i', 'could', 'turn', 'it', 'off', 'i', 'really', 'like', 'this_tv.', 'it', 'take_awhile', 'to', 'find', 'the', 'right', 'setting', 'so', 'it', 'do_not', 'look_wash', 'out']
['energy_saver', 'mode', 'be', 'cause', 'most_of', 'the', 'problem']
['the', 'trumotion', 'setting', 'make', '24fps', 'movie_look', 'like', '30fps', 'home', 'movie', 'otherwise', 'know', 'as', 'soap_opera', 'effect']
['i', 'watch', 'braveheart', 'with', 'it', 'on', 'and', 'it', 'be', 'awful']
['i', 'spend', 'most_of', 'the', 'movie_look', 'through', 'the', 'setting', 'try', 'to', 'find', 'it']
['its', 'in', 'picture', 'options']
['hint', 'turn_off_smart', 'picture_mode', 'and', 'then', 'turn', 'the', 'energy_saver', 'off.', 'with', 'smart', 'picture_mode', 'on', 'and', 'no', 'access', 'to', 'energy_saver', 'setting', 'the', 'picture', 'be', 'very', 'dull', 'and', 'drab']
['energy_saver', 'reduce', 'the', 'backlight', 'to', 'save', 'energy', 'but'

In [224]:
lda = LdaMulticore(trigram_bow, num_topics=5, id2word=trigram_dictionary, workers=2)

In [229]:
def explore_topic(topic_number, topn=25):
    for term, frequency in lda.show_topic(topic_number, topn=25):
        print (term, round(frequency, 3))

In [235]:
explore_topic(0)

expect 0.001
drive 0.001
may_just 0.001
can_not_wait 0.001
also 0.001
store.when 0.001
vesa 0.001
neat 0.001
definition 0.001
or_if 0.001
sit 0.001
engage 0.001
file 0.001
brick 0.001
additional 0.001
my_wife 0.001
liking 0.001
8500 0.001
you_should 0.001
bang_for 0.001
connect 0.001
t.v 0.001
trust 0.001
fire 0.001
straight_from 0.001
